In [0]:
pwd

'/content'

In [0]:
from google.colab import drive
drive.mount('./drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at ./drive


In [0]:
!ls './drive/My Drive/Thesis Work'

 CambridgeMonoPhoneDf.pickle	      jawadmodel2.h5
 CambridgeWordFeaturesDf.pickle       L2monoPhoneDfTest.pickle
 CheckScrapSpeed.ipynb		      L2monoPhoneDfTrain.pickle
 ConvertToWav.ipynb		      L2WordFeaturesDfTest.pickle
'Copy of l2arctic_release_v2.0.zip'   L2WordFeaturesDfTrain.pickle
 Implementation1		      ModelFitting.ipynb
 Implementation2		      MostCommon3000Oxford.txt
 Implementation3		      UKpronunciations
 Implementation4		      USpronunciations


Bism  
# Imports

In [0]:
import pandas as pd
import numpy as np
import os
import IPython.display as ipd
import librosa
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.style.use('ggplot')
from tqdm import tqdm_notebook


#SKLEARN
from sklearn.preprocessing import LabelBinarizer
import sklearn.metrics as sklm

#KERAS
from keras import models
from keras.models import Model, load_model
from keras import callbacks
import keras
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Conv2DTranspose, AveragePooling2D
from keras.optimizers import SGD, Adam

Using TensorFlow backend.


# Implementation 2  (Word Level; Single Phone Mistake)
## L2 as Training set
## L2 as val+test set (utterances held)

### Training

In [0]:
!ls ./drive/My\ Drive/Thesis\ Work/

 CambridgeMonoPhoneDf.pickle	      L2monoPhoneDfTest.pickle
 CambridgeWordFeaturesDf.pickle       L2monoPhoneDfTrain.pickle
 CheckScrapSpeed.ipynb		      L2WordFeaturesDfTest.pickle
 ConvertToWav.ipynb		      L2WordFeaturesDfTrain.pickle
'Copy of l2arctic_release_v2.0.zip'   ModelFitting.ipynb
 Implementation1		      MostCommon3000Oxford.txt
 Implementation2		      UKpronunciations
 jawadmodel2.h5			      USpronunciations


#### X_TRAIN

In [0]:
train_featureDf = pd.read_pickle("./drive/My Drive/Thesis Work/L2WordFeaturesDfTrain.pickle")
train_featureDf['deltaC'] = train_featureDf.MFCC.apply(librosa.feature.delta)
train_featureDf['doubleDeltaC'] = train_featureDf.MFCC.apply(librosa.feature.delta, order=2)

In [0]:
train_featureDf.head()

,word,MFCC,mispronouncedPhones,correctPhones,phoneErrorPercentage,deltaC,doubleDeltaC
0,for,"[[-482.1258206939065, -459.09806101859346, -42...",[],"[F, AO1, R]",0.0,"[[22.44070645615178, 22.44070645615178, 22.440...","[[-5.693856662125653, -5.693856662125653, -5.6..."
1,the,"[[-383.4356920195703, -380.1235950970762, -373...",[],"[DH, AH1]",0.0,"[[-2.1619768795545644, -2.1619768795545644, -2...","[[-2.1235506683236864, -2.1235506683236864, -2..."
2,twentieth,"[[-289.88557478827323, -296.3296463780561, -34...",[],"[T, W, EH1, N, T, IY0, IH0, TH]",0.0,"[[-4.634610784712385, -4.634610784712385, -4.6...","[[12.767461152369506, 12.767461152369506, 12.7..."
3,time,"[[-493.52918905777267, -484.3543551059837, -48...",[],"[T, AY1, M]",0.0,"[[32.19132648271329, 32.19132648271329, 32.191...","[[-2.780640526801415, -2.780640526801415, -2.7..."
4,that,"[[-462.2926228298184, -460.43184706242124, -46...",[],"[DH, AE1, T]",0.0,"[[3.9526547558469938, 3.9526547558469938, 3.95...","[[1.306252970785217, 1.306252970785217, 1.3062..."


In [0]:
train_featureDf = train_featureDf[train_featureDf.phoneErrorPercentage == 0] #only the 100% correct ones for training

In [0]:
train_featureDf.word.shape

(11893,)

In [0]:
x_train = np.array((train_featureDf.MFCC,train_featureDf.deltaC,train_featureDf.doubleDeltaC))
x_train.shape

(3, 11893, 13, 118)

In [0]:
x_train = np.transpose(x_train,[1,2,3,0])
x_train.shape

(11893, 13, 118, 3)

In [0]:
train_words = train_featureDf.word
train_phones = train_featureDf.correctPhones #all correct

In [0]:
np.save('./drive/My Drive/Thesis Work/Implementation2/x_train.npy',x_train) #Feature vector
np.save('./drive/My Drive/Thesis Work/Implementation2/train_words.npy',train_words) #words
np.save('./drive/My Drive/Thesis Work/Implementation2/train_phones.npy',train_phones) #phones per word

#### Y_TRAIN

In [0]:
encoder = LabelBinarizer()
y_train = encoder.fit_transform(train_words)
y_train.shape

(11893, 922)

In [0]:
np.save('./drive/My Drive/Thesis Work/Implementation2/y_train.npy',y_train) #save y_train

#### MODEL

In [0]:
model = Sequential()
# input: 100x100 images with 3 channels -> (100, 100, 3) tensors.
# this applies 32 convolution filters of size 3x3 each.
model.add(Conv2D(32, (3, 3),padding='same', activation='relu', input_shape=(13, 118, 3)))
model.add(Conv2D(32, (3, 3),padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3),padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(922, activation='softmax'))

callbacklist=[callbacks.History(),callbacks.ModelCheckpoint('./jawadmodel.h5', monitor='loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=20)]
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=5000, epochs=150, callbacks=callbacklist)

In [0]:
model.fit(x_train, y_train, batch_size=5000, epochs=60, callbacks=callbacklist)

Epoch 1/60
11893/11893 [==============================] - 4s 368us/step - loss: 0.0193 - acc: 0.9994
Epoch 2/60
11893/11893 [==============================] - 4s 368us/step - loss: 0.0192 - acc: 0.9994
Epoch 3/60
11893/11893 [==============================] - 5s 390us/step - loss: 0.0191 - acc: 0.9994
Epoch 4/60
11893/11893 [==============================] - 5s 387us/step - loss: 0.0191 - acc: 0.9994
Epoch 5/60
11893/11893 [==============================] - 4s 374us/step - loss: 0.0189 - acc: 0.9994
Epoch 6/60
11893/11893 [==============================] - 5s 387us/step - loss: 0.0188 - acc: 0.9994
Epoch 7/60
11893/11893 [==============================] - 5s 387us/step - loss: 0.0187 - acc: 0.9994
Epoch 8/60
11893/11893 [==============================] - 4s 374us/step - loss: 0.0186 - acc: 0.9994
Epoch 9/60
11893/11893 [==============================] - 4s 371us/step - loss: 0.0185 - acc: 0.9994
Epoch 10/60
11893/11893 [==============================] - 4s 375us/step - loss: 0.0183 - a

In [0]:
!cp ./jawadmodel.h5 "./drive/My Drive/Thesis Work/Implementation2/model.h5"

In [0]:
!ls "./drive/My Drive/Thesis Work/Implementation2/"

model.h5		       test_words.npy	 train_words.npy  x_train.npy
test_phoneErrorPercentage.npy  train_phones.npy  x_test.npy	  y_train.npy


Training Completed. Accuracy 100% and Loss 0.0147. 

### TESTING

#### Representation Layer, L2 norm, Prediction Checks

In [0]:
!ls './drive/My Drive/Thesis Work/WordLevel_RL_MP_MFCC_TrainL2_TestL2'

model.h5		       test_words.npy	 train_words.npy  x_train.npy
test_phoneErrorPercentage.npy  train_phones.npy  x_test.npy	  y_train.npy


In [0]:
model = load_model('./drive/My Drive/Thesis Work/WordLevel_RL_MP_MFCC_TrainL2_TestL2/model.h5')

In [0]:
x_train = np.load("./drive/My Drive/Thesis Work/WordLevel_RL_MP_MFCC_TrainL2_TestL2/x_train.npy")
y_train = np.load("./drive/My Drive/Thesis Work/WordLevel_RL_MP_MFCC_TrainL2_TestL2/y_train.npy")

In [0]:
train_words = np.load("./drive/My Drive/Thesis Work/WordLevel_RL_MP_MFCC_TrainL2_TestL2/train_words.npy") #vocabulary

In [0]:
encoder = LabelBinarizer()
encoder.fit(train_words)

LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)

In [0]:
encoder.inverse_transform(model.predict(x_train[:4])) #predicting correctly

array(['for', 'the', 'twentieth', 'time'], dtype='<U14')

In [0]:
encoder.inverse_transform(y_train[:4])

array(['for', 'the', 'twentieth', 'time'], dtype='<U14')

In [0]:
np.all(encoder.inverse_transform(y_train) == train_words) #decoder also working fine

True

In [0]:
model.layers[-2] #representation

In [0]:
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer(index=-2).output)

In [0]:
np.linalg.norm(intermediate_layer_model.predict(x_train[1:2]) - intermediate_layer_model.predict(x_train[1:2])) #l2 norm

0.0

#### Loading Train-Test Arrays

In [0]:
x_test = np.load('./drive/My Drive/Thesis Work/Implementation2/x_test.npy')
test_words = np.load('./drive/My Drive/Thesis Work/Implementation2/test_words.npy')
test_phoneErrorPercentage = np.load('./drive/My Drive/Thesis Work/Implementation2/test_phoneErrorPercentage.npy')
test_mispronouncedPhones = np.load('./drive/My Drive/Thesis Work/Implementation2/test_mispronouncedPhones.npy')
test_correctPhones = np.load('./drive/My Drive/Thesis Work/Implementation2/test_correctPhones.npy')

In [0]:
x_test.shape

(9140, 13, 118, 3)

In [0]:
x_train = np.load("./drive/My Drive/Thesis Work/Implementation2/x_train.npy")
y_train = np.load("./drive/My Drive/Thesis Work/Implementation2/y_train.npy")
train_words = np.load("./drive/My Drive/Thesis Work/Implementation2/train_words.npy") #vocabulary
model = load_model('./drive/My Drive/Thesis Work/Implementation2/model.h5')

#### Validation and Testing

In [0]:
model.layers[-2]

In [0]:
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer(index=-2).output)

In [0]:
#Number of Words not present in Train Set --> These will be skipped in validation and testing
len(set(test_words)) - len(set(test_words).intersection(set(train_words)))

113

In [0]:
len(set(test_words))

262

MORE THAN HALF NOT EVEN PRESENT!!! :O

In [0]:
encoder = LabelBinarizer()
encoder.fit(train_words)

LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)

In [0]:
len(test_words)

9140

In [0]:
train_representations = intermediate_layer_model.predict(x_train)

In [0]:
train_representations.shape

(11893, 64)

In [0]:
#VALIDATION

valSetLength = 1000 #initial some from test
tolerancePercentage = 40

truelabels = [1 if test_phoneErrorPercentage[i] <= tolerancePercentage else 0 for i in range(valSetLength) if test_words[i] in train_words]

accs = []
f1mis = []
for threshold in [150,160,170,180,190,200,210,220,250]:
  predictedLabels = []
  for i in range(valSetLength): #iterating on test words
    if test_words[i] in train_words:
      '''print(i)
      print(test_words[i])
      print(test_phoneErrorPercentage[i])
      print(np.mean(np.linalg.norm(train_representations[np.argwhere(train_words == test_words[i]).reshape(-1,)]\
                       - intermediate_layer_model.predict(x_test[i:i+1]),axis=1)))'''
      if np.mean(np.linalg.norm(train_representations[np.argwhere(train_words == test_words[i]).reshape(-1,)]\
                       - intermediate_layer_model.predict(x_test[i:i+1]),axis=1)) <= threshold:
        predictedLabels.append(1)
      else: 
        predictedLabels.append(0)

  accs.append(sklm.accuracy_score(truelabels,predictedLabels))
  f1mis.append(sklm.precision_recall_fscore_support(truelabels,predictedLabels)[2][0])

print("Threshold: ", np.argmax(f1mis))
print("Max Achievable Accuracy: ", accs[np.argmax(f1mis)])
print("Max Achievable F-1 score for Mispronunciations: ", max(f1mis))

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Threshold:  6
Max Achievable Accuracy:  0.14496644295302014
Max Achievable F-1 score for Mispronunciations:  0.24615384615384614


In [0]:
sklm.precision_recall_fscore_support(truelabels,predictedLabels) #0 - 1

(array([0.14      , 0.86666667]),
 array([0.94230769, 0.06084243]),
 array([0.24378109, 0.11370262]),
 array([104, 641]))

**Testing**

In [0]:
valSetLength = 1000 #initial some from test
tolerancePercentage = 40
threshold = 210
truelabels = [1 if test_phoneErrorPercentage[i] <= tolerancePercentage else 0 for i in range(valSetLength,test_words.shape[0]) if test_words[i] in train_words]

predictedLabels = []
for i in range(valSetLength,test_words.shape[0]): #iterating on test words
  if test_words[i] in train_words:
    '''print(i)
    print(test_words[i])
    print(test_phoneErrorPercentage[i])
    print(np.mean(np.linalg.norm(train_representations[np.argwhere(train_words == test_words[i]).reshape(-1,)]\
                     - intermediate_layer_model.predict(x_test[i:i+1]),axis=1)))'''
    if np.mean(np.linalg.norm(train_representations[np.argwhere(train_words == test_words[i]).reshape(-1,)]\
                     - intermediate_layer_model.predict(x_test[i:i+1]),axis=1)) <= threshold:
      predictedLabels.append(1)
    else: 
      predictedLabels.append(0)

In [0]:
print('Accuracy = ', sklm.accuracy_score(truelabels,predictedLabels))

Accuracy =  0.13755135579293346


In [0]:
sklm.precision_recall_fscore_support(truelabels,predictedLabels) #0 - 1

(array([0.13305785, 0.91428571]),
 array([0.99628713, 0.00606405]),
 array([0.23476232, 0.01204819]),
 array([ 808, 5277]))

In [0]:
sklm.precision_recall_fscore_support(truelabels,predictedLabels)[2][0] #0

0.2347623213764946

In [0]:
sklm.roc_curve(truelabels,predictedLabels)

(array([0.        , 0.00371287, 1.        ]),
 array([0.        , 0.00606405, 1.        ]),
 array([2, 1, 0]))

*CANNOT COMPARE RESULTS:*  
The Interspeech one has a **different dataset** and shows results on phone level (as every phone is a separate class there)

The l2-arctic one shows no results

I am the first one to show results on this dataset